In [ ]:
from operator import indexOf
import torch
import torch.nn.functional as F 
from torch import log_softmax, sigmoid

"""
伪代码：DPO 训练流程
目标：基于偏好数据集优化策略模型，使其更倾向于生成优选响应 y_w 而非拒绝响应 y_l
输入：
  - dataset: 偏好数据集，格式为 [(prompt, chosen_response, rejected_response)]
  - policy_model: 策略模型 π_θ（如 Llama 3.1），待优化
  - reference_model: 参考模型 π_ref（如 SFT 模型），通常固定
  - beta: 温度参数，控制 DPO 损失尺度（通常 0.1–0.5）
  - label_smoothing: 标签平滑参数，引入不确定性（默认 0.0 表示原始 DPO）
  - reference_free: 是否忽略参考模型（默认 False）
  - batch_size: 批次大小
  - max_seq_len: 最大序列长度（例如 Llama 3.1 的 128k token）
输出：
  - losses: 每个样本的 DPO 损失
  - chosen_rewards, rejected_rewards: 优选和拒绝响应的奖励值（用于监控）
"""

def dpo_training(dataset, policy_model, reference_model, beta=0.1, label_smoothing=0.0, reference_free=False, batch_size=32, max_seq_len=512):
    """DPO训练主函数，优化策略模型以对齐人类偏好"""
    # 初始化优化器（例如 AdamW）
    optimizer = torch.optim.AdamW(policy_model.parameters(), lr=1e-5)
    
    # 初始化损失列表和奖励值
    all_losses = []
    all_chosen_rewards = []
    all_rejected_rewards = []
    # 按批次处理数据集
    for batch_idx, batch in enumerate(dataset.batch_iter(batch_size)):
        # 提取批次数据：提示、优选响应、拒绝响应
        # 假设 dataset 已预处理为 token ID 格式
        prompts, chosen_responses, rejected_responses = batch
        # prompts: (batch_size, prompt_len)，token ID
        # chosen_responses: (batch_size, chosen_len)，优选响应 token ID
        # rejected_responses: (batch_size, rejected_len)，拒绝响应 token ID

        # 创建掩码，标记有效 token（1 表示有效，0 表示填充）
        # 假设序列已填充到 max_seq_len
        prompt_mask = (prompts != pad_token_id).float()  # pad_token_id 由分词器定义
        chosen_mask = (chosen_responses != pad_token_id).float()
        rejected_mask = (rejected_responses != pad_token_id).float()

        # 1. 计算对数概率（使用 Teaching Forcing）
        # Teaching Forcing：输入真实序列 y[:t-1]，预测 y[t] 的概率
        def compute_logprobs(model, prompt, response, mask):
            """
            计算序列的对数概率。
            Args:
                model: 策略或参考模型
                prompt: 输入提示，形状 (batch_size, prompt_len)
                response: 目标响应，形状 (batch_size, response_len)
                mask: 掩码，形状 (batch_size, response_len)
            Returns:
                logprobs: 每句话的对数概率和，形状 (batch_size,)
            """
            # 拼接输入：prompt + response[:t-1]
            # 假设模型支持拼接输入，实际中可能需分词器处理
            input_ids = torch.cat([prompt, response[:, :-1]], dim=-1)
            # input_ids: (batch_size, prompt_len + response_len - 1)

            # 前向传播，获取 logits
            # 因果掩码（Causal Mask）由 Transformer 自动应用，确保自回归
            logits = model(input_ids)  # (batch_size, seq_len, vocab_size)

            # 位移操作：对齐 logits 和 labels
            labels = response[:, 1:].clone()  # 去掉第一个 token（如 <bos>）
            logits = logits[:, :-1, :]        # 去掉最后一个 logits

            # 计算对数概率
            logps = log_softmax(logits, dim=-1)  # (batch_size, seq_len-1, vocab_size)

            # 提取目标 token 的对数概率
            select_logprobs = torch.gather(
                input=logps,
                dim=-1,
                index=labels.unsqueeze(-1)
            ).squeeze(-1)  # (batch_size, seq_len-1)

            # 应用掩码，忽略填充 token
            mask = mask[:, 1:].clone()  # 对齐位移
            select_logprobs = select_logprobs * mask
            # 计算序列对数概率和（等价于概率连乘的对数）
            logprobs = select_logprobs.sum(-1)  # (batch_size,)
            # 归一化（可选）：除以有效 token 数量
            # logprobs = select_logprobs.sum(-1) / mask.sum(-1)
            return logprobs

        # 计算策略模型的对数概率
        policy_chosen_logps = compute_logprobs(
            policy_model, prompts, chosen_responses, chosen_mask
        )  # log π_θ(y_w|x)
        policy_rejected_logps = compute_logprobs(
            policy_model, prompts, rejected_responses, rejected_mask
        )  # log π_θ(y_l|x)

        # 计算参考模型的对数概率（若非 reference_free）
        if reference_free:
            reference_chosen_logps = torch.zeros_like(policy_chosen_logps)
            reference_rejected_logps = torch.zeros_like(policy_rejected_logps)
        else:
            reference_chosen_logps = compute_logprobs(
                reference_model, prompts, chosen_responses, chosen_mask
            )  # log π_ref(y_w|x)
            reference_rejected_logps = compute_logprobs(
                reference_model, prompts, rejected_responses, rejected_mask
            )  # log π_ref(y_l|x)

        # 2. 计算 DPO 损失
        # 对数概率比
        pi_logratios = policy_chosen_logps - policy_rejected_logps
        # π_logratios = log (π_θ(y_w|x) / π_θ(y_l|x))
        ref_logratios = reference_chosen_logps - reference_rejected_logps
        # ref_logratios = log (π_ref(y_w|x) / π_ref(y_l|x))

        # 逻辑值（Logits）
        logits = pi_logratios - ref_logratios
        # logits = log (π_θ(y_w|x) / π_θ(y_l|x)) - log (π_ref(y_w|x) / π_ref(y_l|x))

        # Sigmoid 损失
        losses = (
            -F.logsigmoid(beta * logits) * (1 - label_smoothing)
            - F.logsigmoid(-beta * logits) * label_smoothing
        )
        # losses: (batch_size,)，每个样本的 DPO 损失
        # -log_sigmoid(z) = log(1 + e^{-z})，鼓励 logits > 0（即 y_w 更可能）

        # 3. 计算奖励（用于监控，非必需）
        # 奖励定义：r(x, y) = β * log (π_θ(y|x) / π_ref(y|x))
        chosen_rewards = beta * (policy_chosen_logps - reference_chosen_logps)
        rejected_rewards = beta * (policy_rejected_logps - reference_rejected_logps)

        # 4. 优化
        # 计算批量损失均值
        loss = losses.mean()
        # 反向传播
        loss.backward()
        # 更新参数
        optimizer.step()
        # 清空梯度
        optimizer.zero_grad()

        # 记录损失和奖励
        all_losses.append(losses.detach())
        all_chosen_rewards.append(chosen_rewards.detach())
        all_rejected_rewards.append(rejected_rewards.detach())

    # 返回所有损失和奖励
    return torch.cat(all_losses), torch.cat(all_chosen_rewards), torch.cat(all_rejected_rewards)

# 辅助函数：数据预处理（示例）
def preprocess_batch(dataset, batch_size, max_seq_len, tokenizer):
    """
    将原始数据转换为 token ID 和掩码。
    Args:
        dataset: 原始偏好数据集 [(prompt_text, chosen_text, rejected_text)]
        batch_size: 批次大小
        max_seq_len: 最大序列长度
        tokenizer: 分词器（如 Llama 的 tokenizer）
    Yields:
        prompts, chosen_responses, rejected_responses: token ID 和掩码
    """
    for batch in dataset.batch_iter(batch_size):
        prompt_texts, chosen_texts, rejected_texts = zip(*batch)
        
        # 转换为 token ID
        prompt_inputs = tokenizer(prompt_texts, padding=True, truncation=True, 
                                 max_length=max_seq_len, return_tensors="pt")
        chosen_inputs = tokenizer(chosen_texts, padding=True, truncation=True, 
                                  max_length=max_seq_len, return_tensors="pt")
        rejected_inputs = tokenizer(rejected_texts, padding=True, truncation=True, 
                                    max_length=max_seq_len, return_tensors="pt")
        
        # 获取 token ID 和掩码
        prompts = prompt_inputs["input_ids"]  # (batch_size, prompt_len)
        prompt_mask = prompt_inputs["attention_mask"]  # (batch_size, prompt_len)
        chosen_responses = chosen_inputs["input_ids"]
        chosen_mask = chosen_inputs["attention_mask"]
        rejected_responses = rejected_inputs["input_ids"]
        rejected_mask = rejected_inputs["attention_mask"]

        yield (prompts, chosen_responses, rejected_responses), \
              (prompt_mask, chosen_mask, rejected_mask)

# 示例调用
# 假设 dataset 是 [(prompt, chosen, rejected)] 格式
# tokenizer 是 Llama 3.1 的分词器
# policy_model 和 reference_model 是 Transformer 模型
losses, chosen_rewards, rejected_rewards = dpo_training(
    dataset=preprocess_batch(raw_dataset, batch_size=32, max_seq_len=512, tokenizer=tokenizer),
    policy_model=policy_model,
    reference_model=reference_model,
    beta=0.1,
    label_smoothing=0.0,
    reference_free=False
)
    